In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
def MSIS(a,f,uf,lf,obs,m,alpha=.05):
    avger = 1 / len(a)
    num1 = uf - lf
    num2 = np.array([(
        (2/alpha * (lfs - acs)) if lfs > acs else
        (2/alpha * (acs - ufs)) if acs > ufs else
        0
    ) for acs, ufs, lfs in zip(a,uf,lf)])
    davger = 1 / (len(obs) - m)
    denom = np.sum(np.abs(pd.Series(obs).diff(m).values[m:]))
    return avger * (np.sum(num1 + num2) / (davger * denom))

def ACD(a,uf,lf,alpha=.05):
    return np.mean(
                [
                    1 if (
                        acs > ufs
                    ) | (
                        acs < lfs
                    ) else 0 for acs, ufs, lfs in zip(
                        a,uf,lf
                    )
                ]
        )

In [3]:
##### test ######
a = np.array([1,2,3,4])
f = np.array([1,2.5,3.2,4.1])
uf = np.array([4,5,6,7])
lf = np.array([-3,-2,-1,0])
obs = np.array([1,3,4,5,6,67,87,8,9])
m = 2
alpha = .95
MSIS(a,f,uf,lf,obs,m,alpha)

0.17073170731707318

In [4]:
ACD(a,uf,lf)

0.0

In [5]:
mmap = {
    'Yearly':1,
    'Quarterly':4,
    'Monthly':12,
    'Weekly':1,
    'Daily':1,
    'Hourly':24,
}

info = pd.read_csv(
    'm4/M4-info.csv',
    index_col=0,
    parse_dates = ['StartingDate'],
    dayfirst=True,
)
train = {
    freq:pd.read_csv(
        f'm4/train/{freq}-train.csv',
        index_col=0,
    ) for freq in mmap
}
test = {
    freq:pd.read_csv(
        f'm4/test/{freq}-test.csv',
        index_col=0,
    ) for freq in mmap
}

In [6]:
ready = ('Yearly','Quarterly','Monthly','Weekly','Daily','Hourly')
results = {
    freq:pickle.load(
        open(f'{freq}_results.pckl','rb')
    ) for freq in mmap if freq in ready
}

In [7]:
ffa = {
    freq:[
        (
            test[freq].loc[i].dropna().values, # actuals
            *f.export(
                'lvl_fcsts',
                models='top_1',
                determine_best_by='LevelTestSetMAPE',
                cis = True,
            ).iloc[:,1:].T.values, # predictions + cis
            train[freq].loc[i].dropna().values, # obs
        ) for i, f in d.items()
    ] for freq, d in results.items()
}

In [8]:
msis_st1 = (
    [
        [
            MSIS(*af,m=mmap[freq]) for af in ffa[freq]
        ] for freq in ffa
    ] 
)

msis = np.mean(
    [i for sub in msis_st1 for i in sub]
)

In [9]:
msis

37.41370902689927

In [10]:
acd_st1 = (
    [
        [
            ACD(af[0],af[2],af[3]) for af in ffa[freq]
        ] for freq in ffa
    ] 
)

acd = np.mean(
    [i for sub in acd_st1 for i in sub]
)

In [11]:
acd

0.3219214935462449

## Reeval